In [ ]:
#H03T04
#päästöjen summaaminen

In [164]:
def emissions(url):
    import pandas as pd
    
    #tehdään taas frame huonosti muotoillusta tiedostosta
    df = pd.read_csv(url, sep='\t', engine='python')
    sarake = 'country\tcountry_code\tETS information\tmain activity sector name\tunit\tvalue\tyear'
    splittaus = df[sarake].str.split('\t').str
    uusi_df_pohja = {'main activity sector': splittaus[3], 'value' : splittaus[5], 'year' : splittaus[6]}
    df1 = pd.DataFrame(uusi_df_pohja)
    
    #poistetaan halutut rivit
    df1.drop(df1.loc[df1['main activity sector'] == '20-99 All stationary installations'].index, inplace=True)
    df1.drop(df1.loc[df1['main activity sector'] == '21-99 All industrial installations (excl. combustion)'].index, inplace=True)
    
    #poimitaan uuteen frameen halutut vuodet
    #aluksi sarake täytyy muuttaa numeeriseksi että vertailu onnistuu
    df1['year'] = pd.to_numeric(df1['year'], errors = 'coerce')
    df2 = df1.loc[df1['year'] >= 2010]
    df2 = df2.loc[df1['year'] <= 2015]
    
    #lasketaan eri päästölähteiden summat uuteen frameen
    df2['value'] = pd.to_numeric(df2['value'], errors = 'coerce')
    df3 = df2.groupby(['main activity sector'], as_index=False)['value'].sum()
    df3.sort_values('value', ascending=False, inplace=True)
    
    #vaadittua tulostusta varten nimetään sarake uudelleen
    df3.rename(columns = {'main activity sector': 'sector'}, inplace=True)
    
    #automaattitarkistusta varten pitää saada vielä eksponenttimuotoon
    df3['value'] = df3['value'].map('{:e}'.format)
    
    #ja muotoillaan lopullinen tuloste
    tulos = df3[['value','sector']].head(5)
    
    return tulos

In [165]:
#url="https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta3/harjoitukset/ETS_Database_v38A.csv"
url="https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta3/harjoitukset/ETS_Database_v38B.csv"
print(emissions(url))

           value                               sector
1   1.674428e+10               20 Combustion of fuels
5   2.135161e+09  24  Production of pig iron or steel
10  1.859209e+09      29 Production of cement clinker
2   1.714291e+09          21  Refining of mineral oil
0   6.699978e+08                          10 Aviation
